<a href="https://colab.research.google.com/github/gudzen2517/Detector/blob/master/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rake_nltk

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from collections import defaultdict

import nltk
import pandas as pd
from nltk.corpus import stopwords
from rake_nltk import Rake
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

#Rake

In [ ]:
def calculate_word_scores(phrases):
    word_f = defaultdict(int)
    word_d = defaultdict(int)

    for phrase in phrases:
        for word in phrase:
            word_f[word] += 1
            if word_f[word] == 1:
              word_d[word] += len(phrase) - 1

    for word in word_d:
        word_d[word] += word_f[word]

    word_score = {word: word_d[word] / word_f[word] for word in word_f}
    return word_score

def single_score(str_):
    keywords = score(str_)
    keywords_list = [phrase for phrase, score in keywords]
    return keywords_list

def score(msg):
  stop_words = stopwords.words('english')
  row_phrases = re.split(r'[@\"[\-.?!)(,:\]\"]+ ', msg.lower())
  phrases = []
  tmp = []
  for phrase in row_phrases:
    words = re.split(r'\W+', phrase.lower())
    for word in words:
        if word in stop_words or not word or len(tmp) == 3:
            if tmp:
                phrases.append(tmp)
                tmp = []
        else:
            tmp.append(word)
    if phrase:
        phrases.append(tmp)
        tmp = []
  score = calculate_word_scores(phrases)
  phrase_scores = {}
  for phrase in phrases:
      scores = sum(score[word] for word in phrase)
      if scores != 0:
        phrase_scores[" ".join(phrase)] = scores

  return sorted(phrase_scores.items(), key=lambda x: x[1], reverse=True)

# [[[]]]


In [ ]:
def ready_data():
  data = pd.read_csv('spam.csv', encoding='latin-1')
  data = data[['v1', 'v2']]
  data.columns = ['label', 'message']
  my_results = {'Label': [], 'Keywords': []}


  for index, row in data.iterrows():
    keywords = score(row['message'])
    kek = Rake(min_length=1, max_length=3)
    kek.extract_keywords_from_text(row['message'])
    kok = kek.get_ranked_phrases_with_scores()
    print(keywords)
    print(kek.get_ranked_phrases_with_scores())
    print("--------------------------------")
    keywords_list = [phrase for phrase in keywords]
    my_results['Label'].append(row['label'])
    my_results['Keywords'].append(kok)
  keywords_df = pd.DataFrame(my_results)
  keywords_df.to_csv('my_keywords_output.csv', index=False)
  return keywords_df
s = ready_data()

Выходные данные были обрезаны до нескольких последних строк (5000).
[(4.0, 'take care')]
--------------------------------
[('fat head', 4.0), ('means', 1.0)]
[(4.0, 'fat head'), (1.0, 'means')]
--------------------------------
[('sounds like', 4.0), ('still cold', 3.0), ('plan', 1.0), ('cardiff', 1.0), ('still', 1.0), ('sitting', 1.0), ('radiator', 1.0)]
[(4.0, 'sounds like'), (3.5, 'still cold'), (1.5, 'still'), (1.0, 'sitting'), (1.0, 'radiator'), (1.0, 'plan'), (1.0, 'cardiff')]
--------------------------------
[('like proper tongued', 9.0), ('serious', 1.0)]
[(9.0, 'like proper tongued'), (1.0, 'serious')]
--------------------------------
[('wont say hi', 9.0), ('long distance', 4.0), ('good', 1.0), ('wondering', 1.0), ('smiling', 1.0), ('coping', 1.0)]
[(9.0, 'wont say hi'), (4.0, 'long distance'), (1.0, 'wondering'), (1.0, 'smiling'), (1.0, 'good'), (1.0, 'coping')]
--------------------------------
[('da car', 4.0), ('c lar', 4.0), ('wearing shorts', 4.0), ('noe', 1.0), ('later',

#Metrics

In [ ]:
import math
from collections import defaultdict, Counter
import pandas as pd
from rake_nltk import Rake
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def compute_tf(document):
    words = document.split()
    total_words = len(words)
    word_counts = Counter(words)
    return {word: count / total_words for word, count in word_counts.items()}


def compute_idf(corpus):
    total_documents = len(corpus)
    document_frequencies = defaultdict(int)
    for document in corpus:
        unique_terms = set(document.split())
        for term in unique_terms:
            document_frequencies[term] += 1
    return {term: math.log(total_documents / (1 + freq)) for term, freq in document_frequencies.items()}


def compute_tfidf(corpus):
    idf = compute_idf(corpus)
    tfidf_scores = []
    for document in corpus:
        tf = compute_tf(document)
        tfidf = {term: tf[term] * idf[term] for term in tf}
        tfidf_scores.append(tfidf)
    return tfidf_scores


def compute_single_idf(document):
    words = document.split()
    total_words = len(words)
    if total_words == 0:
        return {}
    word_counts = Counter(words)
    idf = {word: math.log(1 + total_words / (count + 1)) for word, count in word_counts.items()}

    return idf
def compute_single_tf_idf(document):
    tf = compute_tf(document)
    idf = compute_single_idf(document)
    tf_idf = {word: tf[word] * idf[word] for word in tf}
    return tf_idf


def single_str_tf_idf(document):
    keywords = single_score(document)
    processed_text = " ".join(value for key, value in keywords)
    tf_idf_scores = compute_single_tf_idf(processed_text)
    return tf_idf_scores

def my_tf_idf():
    df = ready_data()
    corpus = []
    for phrase in df['Keywords'].to_list():
      corpus.append(" ".join(value for key, value in phrase))
    tfidf_scores = compute_tfidf(corpus)
    tfidf_df = pd.DataFrame(tfidf_scores).fillna(0)
    tfidf_df['label'] = df['Label']
    return tfidf_df

def single_str_tf_idf(document):
    keywords = single_score(document)
    processed_text = " ".join(keywords)
    tf_idf_scores = compute_single_tf_idf(processed_text)
    return tf_idf_scores

my_tf_idf()

Выходные данные были обрезаны до нескольких последних строк (5000).
[(4.0, 'take care')]
--------------------------------
[('fat head', 4.0), ('means', 1.0)]
[(4.0, 'fat head'), (1.0, 'means')]
--------------------------------
[('sounds like', 4.0), ('still cold', 3.0), ('plan', 1.0), ('cardiff', 1.0), ('still', 1.0), ('sitting', 1.0), ('radiator', 1.0)]
[(4.0, 'sounds like'), (3.5, 'still cold'), (1.5, 'still'), (1.0, 'sitting'), (1.0, 'radiator'), (1.0, 'plan'), (1.0, 'cardiff')]
--------------------------------
[('like proper tongued', 9.0), ('serious', 1.0)]
[(9.0, 'like proper tongued'), (1.0, 'serious')]
--------------------------------
[('wont say hi', 9.0), ('long distance', 4.0), ('good', 1.0), ('wondering', 1.0), ('smiling', 1.0), ('coping', 1.0)]
[(9.0, 'wont say hi'), (4.0, 'long distance'), (1.0, 'wondering'), (1.0, 'smiling'), (1.0, 'good'), (1.0, 'coping')]
--------------------------------
[('da car', 4.0), ('c lar', 4.0), ('wearing shorts', 4.0), ('noe', 1.0), ('later',

,jurong,point,crazy,..,go,available,ok,lar,...,std,...,heap,lowes,postcode,087187272008,now1,suggestions,pity,acted,bitching,label
0,1.32206,0.997742,1.010093,0.505133,0.562206,0.946845,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ham
1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.044149,1.911713,0.800978,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ham
2,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.36633,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,spam
3,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.480587,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ham
4,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ham
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.528824,0.528824,0.000000,0.000000,0.000000,0.000000,spam
5568,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,ham
5569,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600733,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,1.983091,1.983091,0.000000,0.000000,ham
5570,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.991545,0.991545,ham


#Fit

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset

data = pd.read_csv('spam.csv', encoding='latin-1')
data = data[['v1', 'v2']]
data.columns = ['label', 'message']

tfidf_df = my_tf_idf()
label_encoder = LabelEncoder()
tfidf_df['label_encoded'] = label_encoder.fit_transform(tfidf_df['label'])
X = tfidf_df.drop(columns=['label', 'label_encoded']).values
y = tfidf_df['label_encoded'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


class SpamClassifier(nn.Module):
    def __init__(self, input_size):
        super(SpamClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x


input_size = X_train.shape[1]
model = SpamClassifier(input_size)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def evaluate_model(model, test_loader, test_data):
    model.eval()
    correct = 0
    total = 0
    predictions_list = []
    actual_labels = []
    keywords_list = []
    correct_predictions = []

    with torch.no_grad():
        for i, (X_batch, y_batch) in enumerate(test_loader):
            outputs = model(X_batch)
            predictions = (outputs > 0.5).float()
            correct += (predictions == y_batch).sum().item()
            total += y_batch.size(0)
            predictions_list.extend(predictions.cpu().numpy().flatten())
            actual_labels.extend(y_batch.cpu().numpy().flatten())
            keywords_list.extend(
                test_data['message'].iloc[i * test_loader.batch_size:(i + 1) * test_loader.batch_size].values)
            correct_predictions.extend(predictions.cpu().numpy().flatten() == y_batch.cpu().numpy().flatten())

    accuracy = correct / total
    print(f"Accuracy on test set: {accuracy * 100:.2f}%")

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}")



train_model(model, train_loader, criterion, optimizer, epochs=10)
evaluate_model(model, test_loader, data.iloc[X_train.shape[0]:])

all_features = tfidf_df.drop(columns=['label', 'label_encoded']).columns.tolist()
def vectorize_single_input(single_tf_idf_result, all_features):
    feature_vector = [single_tf_idf_result.get(word, 0) for word in all_features]
    return feature_vector

def evaluate_single_input(model, text, all_features):
    tf_idf_vector = single_str_tf_idf(text)
    feature_vector = vectorize_single_input(tf_idf_vector, all_features)
    input_tensor = torch.tensor([feature_vector], dtype=torch.float32)
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        print(output)
        prediction = (output > 0.5).float().item()
        print(prediction)

    label = "Spam" if prediction == 1.0 else "Not Spam"
    print(f"Text: {text}")
    print(f"Prediction: {label} (Model output: {output.item():.4f})")


example_text = r"They treat me like aids patent"
evaluate_single_input(model, example_text, all_features)

example_text = r"Congratulations! Click here to claim your reward."
evaluate_single_input(model, example_text, all_features)

Выходные данные были обрезаны до нескольких последних строк (5000).
--------------------------------
[('yeah whatever lol', 9.0)]
[(9.0, 'yeah whatever lol')]
--------------------------------
[('accept day u', 9.0), ('brother sister lover', 9.0), ('best1 clos1 lvblefrnd', 9.0), ('cutefrnd lifpartnr belovd', 9.0), ('rply means enemy', 9.0), ('today', 1.0), ('bstfrnd', 1.0)]
[(9.0, 'rply means enemy'), (1.0, 'today')]
--------------------------------
[('ard 530 lor', 8.0), ('message ì_ lor', 8.0), ('ok', 1.0)]
[(9.0, 'message ì_ lor'), (9.0, 'ard 530 lor'), (1.0, 'ok')]
--------------------------------
[('c u', 4.0), ('ok', 1.0)]
[(4.0, 'c u'), (1.0, 'ok')]
--------------------------------
[('eh ur laptop', 9.0), ('say mon muz', 9.0), ('look c got', 9.0), ('stock lei', 4.0), ('take', 1.0)]
[(9.0, 'look c got'), (8.0, 'stock lei ...'), (2.0, '...'), (1.0, 'take')]
--------------------------------
[('ke qi', 4.0), ('bored izzit', 4.0), ('suddenly thk', 4.0), ('need', 1.0), ('ìï', 1.0)]
[(4